# Downloading videos GB1 por Mestre Carlos Gracie Jr. 

### Getting the videos ids

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import json

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def load_cookies(driver, cookies_file):
    with open(cookies_file, 'r', encoding='utf-8') as f:
        cookies = json.load(f)
    for cookie in cookies:
        if 'sameSite' in cookie:
            if cookie['sameSite'] == 'unspecified':
                cookie['sameSite'] = 'None'
            elif cookie['sameSite'] == 'no_restriction':
                cookie['sameSite'] = 'None'
            elif cookie['sameSite'] == 'lax':
                cookie['sameSite'] = 'Lax'
            elif cookie['sameSite'] == 'strict':
                cookie['sameSite'] = 'Strict'
        if 'expiry' in cookie:
            cookie['expires'] = cookie.pop('expiry')
        try:
            driver.add_cookie(cookie)
        except Exception as e:
            print(f"Não foi possível adicionar o cookie {cookie['name']}: {e}")

driver.get('https://online.graciebarra.com.br/')
time.sleep(3) 

load_cookies(driver, 'cookies.json')

driver.get('https://online.graciebarra.com.br/')
time.sleep(3)

url = 'https://online.graciebarra.com.br/programs/collection-ds849l4dl38?cid=2990058&permalink=semana-1-aula-a-tcnica-1-46859-b43855'
driver.get(url)

xpath_div = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[2]/div[2]/turbo-frame/div/div[2]'

ids = []

try:
    element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, xpath_div))
    )

    div_element = driver.find_element(By.XPATH, xpath_div)

    items = div_element.find_elements(By.CLASS_NAME, 'contents')

    for item in items:
        inner_item_div = item.find_elements(By.TAG_NAME, 'div')[0]
        if inner_item_div.get_attribute('data-controller'):  
            id = item.get_attribute('id')
            if id:
                ids.append(id.replace('program_playlist_chapter_', ''))
except Exception as e:
    print("Ocorreu um erro:", e)
    
print('IDS: ', ids)

IDS:  ['2990058', '2990059', '2990060', '2990062', '2990063', '2990064', '2990067', '2990068', '2990069', '2990071', '2990072', '2990073', '2990076', '2990077', '2990078', '2990080', '2990081', '2990082', '2990085', '2990086', '2990087', '2990089', '2990090', '2990091', '2990094', '2990095', '2990096', '2990098', '2990099', '2990100', '2990103', '2990104', '2990105', '2990107', '2990108', '2990109', '2990112', '2990113', '2990114', '2990116', '2990117', '2990118', '2990121', '2990122', '2990123', '2990125', '2990126', '2990127', '2990130', '2990131', '2990132', '2990134', '2990135', '2990136', '2990139', '2990140', '2990141', '2990143', '2990144', '2990145', '2990148', '2990149', '2990150', '2990152', '2990153', '2990154', '2990157', '2990158', '2990159', '2990161', '2990162', '2990163', '2990166', '2990167', '2990168', '2990170', '2990171', '2990172', '2990175', '2990176', '2990177', '2990179', '2990180', '2990181', '2990184', '2990185', '2990186', '2990188', '2990189', '2990190', '29

### Downloading videos and annotate descriptions

In [9]:
from yt_dlp import YoutubeDL
import csv

try:
    with open('../dataset/gb1/annotations.csv', mode='w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for id in ids:
            url = 'https://online.graciebarra.com.br/programs/collection-ds849l4dl38?cid=' + id
            driver.get(url)
            
            description_xpath = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[4]/div[1]/turbo-frame/ds-show-more/div'

            description_element = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, description_xpath))
            )
            
            description_text = description_element.text.strip()

            writer.writerow({'id': id, 'description': description_text})
            
            source_xpath = '/html/body/main/div[1]/div/turbo-frame/turbo-frame/turbo-frame/div[2]/div[2]/turbo-frame/div/video-player/video/source'
            
            source = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, source_xpath))
            )
            
            source_src = source.get_attribute('src')
            
            ydl_opts = {
                'outtmpl': f'../dataset/gb1/videos/video_{id}.mp4',  
                'format': 'best',             
                'quiet': False,                
            }
            
            with YoutubeDL(ydl_opts) as ydl:
                ydl.download([source_src])
            
            print(f'Vídeo {id} baixado com sucesso!')
except Exception as e:
    print("Ocorreu um erro:", e)
finally:
    driver.quit()

### Identify positions in video and annotate times

In [4]:
import cv2
import numpy as np
from difflib import SequenceMatcher
import easyocr
import pandas as pd

reader = easyocr.Reader(['pt'])

def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    gray = cv2.medianBlur(gray, 3)

    thresh = cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    altura, largura = thresh.shape
    roi = thresh[int(altura * 0.7):altura, 0:largura]

    return roi

def detectar_tecnica(roi):
    resultados = reader.readtext(roi)
    texto_extraido = ' '.join([res[1] for res in resultados]).strip().lower()
    return texto_extraido

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def detectar_melhor_tecnica(texto_extraido, tecnicas_conhecidas):
    melhor_tecnica = ''
    maior_similaridade = 0

    for tecnica in tecnicas_conhecidas:
        sim = similar(texto_extraido, tecnica.lower())
        if sim > maior_similaridade:
            maior_similaridade = sim
            melhor_tecnica = tecnica

    if maior_similaridade > 0.5:
        print('Técnica detectada: ', melhor_tecnica)
        return melhor_tecnica, maior_similaridade
    else:
        return None, maior_similaridade

df = pd.read_csv('../dataset/gb1/annotations.csv')

for idx, row in df.iterrows():
    video_id = row['id']
    print('Processando vídeo', video_id)
    description = row['description']
    tecnicas = description.split('Destaques da Técnica')[1].split('\n')[1:]
    print('Técnicas: ', tecnicas)
    
    cap = cv2.VideoCapture(f'../dataset/gb1/videos/video_{video_id}.mp4')

    detected_techniques = {}
    frame_count = 0
    fps = cap.get(cv2.CAP_PROP_FPS)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        roi = process_frame(frame)

        texto_extraido = detectar_tecnica(roi)
        melhor_tecnica, similaridade = detectar_melhor_tecnica(texto_extraido, tecnicas)

        if melhor_tecnica:
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            if melhor_tecnica not in detected_techniques:
                detected_techniques[melhor_tecnica] = []
            detected_techniques[melhor_tecnica].append(current_time)

    cap.release()

    for tecnica, tempos in detected_techniques.items():
        df.at[idx, tecnica] = ', '.join(map(str, tempos))

# Salva o DataFrame atualizado no arquivo CSV
df.to_csv('annotations_updated.csv', index=False)

print("Processamento concluído e CSV atualizado salvo como 'annotations_updated.csv'")


Processando vídeo 2990058
Técnicas:  ['Clinch', 'Elevação de Quadril', 'Levantada Técnica']
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Clinch
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Elevação de Quadril
Técnica detectada:  Eleva

/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '27.193833333333338, 28.1281, 28.5285, 28.561866666666667, 28.595233333333336, 28.661966666666668, 28.795433333333335, 28.862166666666667, 29.295933333333334' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, tecnica] = ', '.join(map(str, tempos))
/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '39.70633333333333, 39.7397, 39.77306666666667, 39.806433333333345, 40.04, 40.07336666666667, 40.140100000000004, 40.17346666666667, 40.2402, 40.3403, 40.77406666666667, 40.9409, 40.97426666666667, 41.00763333333334, 41.07436666666667, 41.17446666666667, 41.2412, 41.37466666666667, 41.87516666666667, 42.17546666666667, 42.27556666666667, 42.609233333333336' has dtype incompatible with fl

Processando vídeo 2990059
Técnicas:  ['Domínio de Braço', 'Quebra da Postura', 'Pé no Bíceps']
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada:  Domínio de Braço
Técnica detectada

/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '12.746066666666668, 12.779433333333333, 12.812800000000001, 12.846166666666665, 12.879533333333335, 12.9129, 12.946266666666668, 13.046366666666668, 13.079733333333333, 13.346666666666668, 13.580233333333334, 13.646966666666668, 13.713700000000001, 13.8138, 13.847166666666668, 13.880533333333334, 13.947266666666668, 14.647966666666667, 14.681333333333335, 14.8148, 14.881533333333334, 14.948266666666667, 15.048366666666668, 15.148466666666668, 15.281933333333335, 15.3153, 15.4154, 15.448766666666668, 15.515500000000001, 15.682333333333334, 15.7157' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, tecnica] = ', '.join(map(str, tempos))
/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. 

Processando vídeo 2990060
Técnicas:  ['Oponente Levanta Uma das Pernas', 'Pé no Quadril', 'Apoio do Cotovelo']
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técnica detectada:  Oponente Levanta Uma das Pernas
Técni

...

/tmp/ipykernel_7244/2549325223.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[idx, tecnica] = ', '.join(map(str, tempos))
/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '15.4154, 15.515500000000001, 15.749066666666668, 15.815800000000001, 15.982633333333336, 16.082733333333334, 16.14946666666667, 16.182833333333335, 16.249566666666666, 16.282933333333332, 16.316300000000002, 16.349666666666668, 16.4164, 16.44976666666667, 16.483133333333335, 16.5165, 16.549866666666667, 16.616600000000002, 16.649966666666668, 16.683333333333334, 16.7167, 16.75006666666667, 16.783433333333335, 16.850166666666667, 16.883533333333332, 16.916900000000002, 16.9

Processando vídeo 2990198
Técnicas:  ['Cotovelo Abre Espaço', 'Controle das Costas', 'Apoio do Cotovelo']
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre Espaço
Técnica detectada:  Cotovelo Abre

/tmp/ipykernel_7244/2549325223.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[idx, tecnica] = ', '.join(map(str, tempos))
/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '30.497133333333334, 30.5305, 30.56386666666667, 30.597233333333335, 30.663966666666667, 30.697333333333336, 30.730700000000002, 30.76406666666667, 30.797433333333334, 30.8308, 30.86416666666667, 30.897533333333335, 30.9309, 30.964266666666667, 30.997633333333336, 31.031000000000002, 31.06436666666667, 31.097733333333334, 31.1311, 31.16446666666667, 31.197833333333335, 31.2312, 31.264566666666667, 31.297933333333336, 31.331300000000002, 31.36466666666667, 31.398033333333334

Processando vídeo 2990199
Técnicas:  ['Esconder o Braço', 'Ajoelhar Dando as Costas', 'Reposição de Guarda']
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Técnica detectada:  Esconder o Braço
Téc

/tmp/ipykernel_7244/2549325223.py:88: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[idx, tecnica] = ', '.join(map(str, tempos))
/tmp/ipykernel_7244/2549325223.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '24.591233333333335, 24.691333333333336, 24.724700000000002, 24.758066666666668, 24.891533333333335, 24.958266666666667, 24.991633333333336, 25.025000000000002, 25.058366666666668, 25.1251, 25.15846666666667, 25.191833333333335, 25.2252, 25.258566666666667, 25.291933333333336, 25.325300000000002, 25.358666666666668, 25.392033333333334, 25.45876666666667, 25.492133333333335, 25.5255, 25.558866666666667, 25.592233333333336, 25.625600000000002, 25.69233333333333, 25.7257, 25.7